In [ ]:
!pip install scrapy

In [2]:
import numpy as np
import pandas as pd
import time 
import requests
from scrapy.http import TextResponse 

In [3]:
url = "https://staff.am/en/jobs"
base_url = "https://staff.am"
page = requests.get(url)
response = TextResponse(url=page.url,body=page.text,encoding="utf-8")

In [6]:
# Defining functions for companies, jobs, deadlines, locations, and individual pages
# in order to scrape the first page of the website (used both css and xpath selectors).

def get_company(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  company = response.css("p[class = 'job_list_company_title']::text").extract()
  return company 
  
def get_job(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  job = response.css("p[class = 'font_bold']::text").extract()
  return job 

def get_deadline(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  deadline = response.xpath("//div[@class='job-inner job-list-deadline']/p[1]/text()").extract()
  deadline = [i.replace("\n","") for i in deadline]
  deadline_updated = [i for i in deadline if len(i)>0]
  return deadline_updated

def get_location(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  location = response.css("div[class = 'job-inner job-list-deadline'] > p[class = 'job_location']::text").extract()
  location = [i.replace("\n","") for i in location]
  location_updated = [i for i in location if len(i)>0]
  return location_updated

def get_individual_page(url):
  page = requests.get(url)
  response = TextResponse(url=page.url,body=page.text,encoding="utf-8")
  individual_page = [base_url + i for i in response.css("div[class = 'jobListButtonsBlock job_list_button_block'] > div[class = 'job-inner-right text-right load-more-container pull-right'] > a[class = 'load-more btn width100 job_load_more radius_changes']::attr(href)").extract()]
  return individual_page


In [7]:
# Writing a code all_pages in order to scrape all of the pages.
# The amount of pages is equal to 12.
all_pages = ["https://staff.am/en/jobs?page={}&per-page=50".format(i) for i in range(1,13)]

In [8]:
# Creating empty lists and extending them with first scraped page in order 
# to receive all the necessary data for all pages. 
all_companies = []
all_jobs = []
all_deadlines = []
all_locations = []
all_individual_pages = []
for i in all_pages:
    all_companies.extend(get_company(i))
    all_jobs.extend(get_job(i))
    all_deadlines.extend(get_deadline(i))
    all_locations.extend(get_location(i))
    all_individual_pages.extend(get_individual_page(i))
    time.sleep(1)

In [40]:
# Creating a dataframe and turning it into csv file.
df = pd.DataFrame(np.column_stack([all_companies, all_jobs, all_deadlines, all_locations, all_individual_pages]), columns = ['Companies', 'Jobs', 'Deadline', 'Location', 'Individual_Page'])

In [41]:
 df.to_csv('all_jobs.csv', index=False) 

In [42]:
data = pd.read_csv("all_jobs.csv")

In [43]:
# Question 1. Using value.counts() in order to see the most 
# popular company in terms of job posting count. 
data.Companies.value_counts().head(5)

Digitain              32
SoftConstruct         29
PicsArt               23
ServiceTitan          16
TeamViewer Armenia    12
Name: Companies, dtype: int64

In [50]:
# Question 2. Using str.lower() in order to lowercase the word "data"
# on the website, and value_counts() and sum() in order to see how many jobs 
# have this word in it. 
data[data.Jobs.str.lower().str.contains("data")]["Jobs"].value_counts().sum()

10